# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parent.parent.parent)
os.getcwd()

In [ ]:
import sys
sys.path.append('/home/tamir/workspace/automotive/src/python')

import pandas as pd
import numpy as np

# Running on set

In [ ]:
from services.lanes.src.corridor import LanesCorridor, LanesCorridorParams

def run_on_set(set_path: Path, skip=1, max=-1, plot=False, relevant_images=None, det='gt'):
    if det == 'gt':
        df_path = set_path.joinpath('results/infer/b2b_det.tsv')
        pred_df = pd.read_csv(df_path, sep='\t')
    else:
        df_path = set_path.joinpath('prod_det_sf.tsv')
        pred_df = pd.read_csv(df_path, sep='\t')
        pred_df.name = pred_df.name.apply(lambda x: str(x) + '.png')
    cametra_interface_data_path = set_path
    images_path = set_path.joinpath('images')

    max = len(pred_df.name.unique()[::skip]) if max == -1 else max
    
    relevant_imgs = pred_df.name.unique()[::skip][:max] if relevant_images is None else relevant_images
    pred_df = pred_df[pred_df.name.isin(relevant_imgs)]

    params = LanesCorridorParams(cametra_data_path=cametra_interface_data_path, 
                                  images_path=images_path,
                                  plot=plot)
    rbs = LanesCorridor(params=params)

    df_with_results, lanes_dict = rbs.apply(detections=pred_df, 
                           save_path=df_path.parent)
    
    return df_with_results

In [ ]:
imgs = ['1693509028000146.png', '1693509065000194.png', '1693508981000069.png', '1693508648999816.png', '1693508825000187.png', '1693508971000105.png', '1693508715000040.png']
# imgs = ['1693508648999816.png']

run_on_set(Path('/home/tamir/s3_sync/europe_run_3_10/64f561c6ac243656c2bd4fcd'), plot=True, relevant_images=imgs, det='prod')


In [ ]:
imgs = [
    '1693505384000123.png',
    '1693505127000227.png',
    '1693504169000151.png',
    '1693504920999948.png', 
    '1693504688999997.png', 
    '1693505130000001.png', 
    '1693504325999935.png', 
    '1693504412999993.png', 
    '1693504410000127.png']

run_on_set(Path('/home/tamir/s3_sync/europe_run_3_10/64f56183ac243656c2bd4fbe'), plot=True, relevant_images=imgs, det='prod')

In [ ]:
imgs = [
    '1686906627000293.png'
    ]

run_on_set(Path('/home/tamir/s3_sync/europe_run_3_10/64f45788ac243656c2bd4986'), plot=True, relevant_images=imgs, det='gt')

In [ ]:
import pandas as pd
df = pd.read_csv('/home/tamir/workspace/AB_B2B_Eval/tmp/64f45788ac243656c2bd4986_eval_test/inputs/b2b_det.tsv', sep='\t')
df

In [ ]:
df.lane_assignment

# Creating lanes.json and copy to s3

In [ ]:
super_set_name = "europe_run_3_10"
set_name = "64f45788ac243656c2bd4986"
set_ids = [
    "64f45788ac243656c2bd4986",
    "64f56176ac243656c2bd4fbb",
    "64f56183ac243656c2bd4fbe",
    "64f561c6ac243656c2bd4fcd",
    "64f56167ac243656c2bd4fb7",
    "64f5619dac243656c2bd4fc3",
    "64ff1f1a7edfb609dc1b292a",
    "64f6b55eac243656c2bd5c6e",
    "64f6f556ac243656c2bd606f",
    "65013e677edfb609dc1b3e31",
    "65013e227edfb609dc1b3e22",
    "65013e517edfb609dc1b3e2c",
    "65013e3b7edfb609dc1b3e27"
]
for set_name in set_ids:
    run_lanes_filter_on_set(super_set_name, set_name)

In [ ]:
import subprocess

# Replace the command with your own
set_name = "64f561c6ac243656c2bd4fcd"
for set_name in set_ids:
    json_path = f"/home/tamir/s3_sync/europe_run_3_10/{set_name}/results/infer/lanes.json"
    s3_path = f"s3://b2b-datasets/sets_to_generate_gt_for/europe_run_3_10/{set_name}/eval_on_infer_0.1.4/prod_sf_vs_al/inputs/"
    command = f"aws s3 cp {json_path} {s3_path}"
    print(command)

# # Run the command
# result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# result